<hr>
<br>
<br>
<center><b><big> MODEL SENSITIVITY TESTS </b></center></big>
<br>
<br>
1. Individually run each of the cells to reproduce the synthetic tests of our paper.
<br>
2. Because random noise is added, convergence might vary and produce slightly different results.
<hr>

<hr>
<br>
<br>
<j><b> 1. Importing libraries </b></j>
<br>
<br>
<hr>

In [ ]:
import pandas as pd
import math
import ipywidgets as widgets
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats
import plotly.graph_objects as go
from Hist_functions import GGCD_1C,GGCD_2C,GGCD_3C,ferro_area,rev_ferro_area,region_selection,line_inv,moving_mean,gradient,high_field
from Hist_functions import Levenberg_Marquardt_1C,Levenberg_Marquardt_2C,Levenberg_Marquardt_3C,Levenberg_Marquardt_HF
from IPython.display import display
from scipy.special import gamma
from scipy.special import gammainc
from scipy.integrate import simpson
from scipy import interpolate
from numpy import trapz
from matplotlib.offsetbox import AnchoredText
from mpl_toolkits.mplot3d import axes3d
import matplotlib.gridspec as gridspec

plt.rcParams.update({"text.usetex": True, "font.family": "serif", "font.size": 14})

In [ ]:
#defining some extra functions to generate and analyze synthetic data

def CDF(u,sigma,alfa,beta,I,x):
    bterm=(-1/beta)*np.log10(0.5-((1/np.pi)*np.arctan((x-u)/sigma)))
    cumullative= gammainc(alfa,bterm)*I
    
    return cumullative


def get_axis_limits(ax, scale=.9):
    return ax.get_xlim()[1]*scale, ax.get_ylim()[1]*scale


def R2(y,yc):
    SQR=np.sum((y-yc)**2)
    SQT=np.sum((y-np.mean(y))**2)
    R2=1-(SQR/SQT)
    
    return R2

def as_si(x, ndp):
    s = '{x:0.{ndp:d}e}'.format(x=x, ndp=ndp)
    m, e = s.split('e')
    return r'{m:s}\times 10^{{{e:d}}}'.format(m=m, e=int(e))

<hr>
<br>
<br>
<center><b> FORWARD MODELS </b></center>
<br>
<br>
<hr>

In [ ]:
N=1000 #defining number of points 
noise1=5e-6 #defining intensity of noise 1
noise2=1e-3 #defining intensity of noise 2
noise3=1e-2 #defining intensity of noise 3

#creating each of the synthetic examples

x=np.linspace(-1,1,N)

Bc1,Bc2,Bc3,Bc4,Bc5=0.01,0.08,0.2,0.5,0.7
tet1,tet2,tet3,tet4,tet5=0.1,0.1,0.07,0.3,0.2
alfa1,alfa2,alfa3,alfa4,alfa5=1,1,0.7,0.6,0.3
beta1,beta2,beta3,beta4,beta5=1,0.3,0.1,0.7,0.4
i1,i2,i3,i4,i5=0.1,0.05,0.05,0.1,0.1
X0_1,X0_2,X0_3,X0_4,X0_5=1e-2,0.5e-1,-1.2e-2,-0.5e-1,0

H1_nl=CDF(Bc1,tet1,alfa1,beta1,i1,x)+X0_1
H2_nl=CDF(Bc2,tet2,alfa2,beta2,i2,x)+X0_2
H3_nl=CDF(Bc3,tet3,alfa3,beta3,i3,x)+X0_3
H4_nl=CDF(Bc4,tet4,alfa4,beta4,i4,x)+X0_4
H5_nl=CDF(Bc5,tet5,alfa5,beta5,i5,x)+X0_5


H1=CDF(Bc1,tet1,alfa1,beta1,i1,x)+np.random.normal(0,noise1,np.size(x))
H2=CDF(Bc2,tet2,alfa2,beta2,i2,x)+np.random.normal(0,noise1,np.size(x))
H3=CDF(Bc3,tet3,alfa3,beta3,i3,x)+np.random.normal(0,noise1,np.size(x))
H4=CDF(Bc4,tet4,alfa4,beta4,i4,x)+np.random.normal(0,noise1,np.size(x))
H5=CDF(Bc5,tet5,alfa5,beta5,i5,x)+np.random.normal(0,noise1,np.size(x))


C1=np.gradient(H1,x)
C2=np.gradient(H2,x)
C3=np.gradient(H3,x)
C4=np.gradient(H4,x)
C5=np.gradient(H5,x)

C1_C2=np.gradient((H1_nl+H2_nl+np.random.normal(0,noise1,np.size(x))),x)+X0_1
C1_C3=np.gradient((H1_nl+H3_nl+np.random.normal(0,noise1,np.size(x))),x)+X0_3
C1_C4=np.gradient((H1_nl+H4_nl+np.random.normal(0,noise1,np.size(x))),x)+X0_2
C1_C5=np.gradient((H1_nl+H5_nl+np.random.normal(0,noise1,np.size(x))),x)+X0_3
C2_C3=np.gradient((H2_nl+H3_nl+np.random.normal(0,noise1,np.size(x))),x)+X0_4
C2_C4=np.gradient((H2_nl+H4_nl+np.random.normal(0,noise1,np.size(x))),x)+X0_5
C2_C5=np.gradient((H2_nl+H5_nl+np.random.normal(0,noise1,np.size(x))),x)+X0_5
C3_C4=np.gradient((H3_nl+H4_nl+np.random.normal(0,noise1,np.size(x))),x)+X0_2
C3_C5=np.gradient((H3_nl+H5_nl+np.random.normal(0,noise1,np.size(x))),x)+X0_5
C4_C5=np.gradient((H4_nl+H5_nl+np.random.normal(0,noise1,np.size(x))),x)+X0_4


C1_C3_C5=np.gradient((H1_nl+H3_nl+H5_nl+np.random.normal(0,noise1,np.size(x))),x)+X0_1
C3_C4_C5=np.gradient((H3_nl+H4_nl+H5_nl+np.random.normal(0,noise1,np.size(x))),x)+(X0_4*-1)

#plotting the data

figure,(ax1,ax2,ax3)=plt.subplots(3,1,figsize=(8,13))

ax1.plot(x,C1,label=f'$C_{1}$')
ax1.plot(x,C2,label=f'$C_{2}$')
ax1.plot(x,C3,label=f'$C_{3}$')
ax1.plot(x,C4,label=f'$C_{4}$')
ax1.plot(x,C5,label=f'$C_{5}$')
ax1.grid(alpha=0.5)
ax1.set_ylabel('$\kappa$')
ax1.set_xlabel('Applied field (T)')
ax1.legend(fontsize=11,shadow=True,loc='upper left')

ax2.plot(x,C1_C2,label=f'$C_{1}+C_{2}$')
ax2.plot(x,C1_C3,label=f'$C_{1}+C_{3}$')
ax2.plot(x,C1_C4,label=f'$C_{1}+C_{4}$')
ax2.plot(x,C1_C5,label=f'$C_{1}+C_{5}$')
ax2.plot(x,C3_C5,label=f'$C_{3}+C_{5}$')
ax2.plot(x,C4_C5,label=f'$C_{4}+C_{5}$')
ax2.set_ylabel('$\kappa$')
ax2.set_xlabel('Applied field (T)')
ax2.grid(alpha=0.5)
ax2.legend(fontsize=11,shadow=True,loc='upper left')

ax3.plot(x,C1_C3_C5,label=f'$C_{1}+C_{3}+C_{5}$')
ax3.plot(x,C3_C4_C5,label=f'$C_{3}+C_{4}+C_{5}$')
ax3.set_ylabel('$\kappa$')
ax3.set_xlabel('Applied field (T)')
ax3.grid(alpha=0.5)
ax3.legend(fontsize=11,shadow=True,loc='upper left')


at1 = AnchoredText(
    "a", prop=dict(size=15), frameon=True, loc='upper right')
at1.patch.set_boxstyle("square,pad=0.")
ax1.add_artist(at1)

at2 = AnchoredText(
    "b", prop=dict(size=15), frameon=True, loc='upper right')
at2.patch.set_boxstyle("square,pad=0.")
ax2.add_artist(at2)

at3 = AnchoredText(
    "c", prop=dict(size=15), frameon=True, loc='upper right')
at3.patch.set_boxstyle("square,pad=0.")
ax3.add_artist(at3)

figure.tight_layout()
plt.show()

<hr>
<br>
<br>
<center><b> INVERSE MODELS </b></center>
<br>
<br>
<hr>

<hr>
<br>
<br>
<b> Single ferromagnetic component </b>
<br>
<br>
<hr>

In [ ]:
#initial guesses for parameter I

Iguesser_C1,a1=ferro_area(x,C1,C1,region_selection,line_inv)
Iguesser_C2,a2=ferro_area(x,C2,C2,region_selection,line_inv)
Iguesser_C3,a3=ferro_area(x,C3,C3,region_selection,line_inv)
Iguesser_C4,a4=ferro_area(x,C4,C4,region_selection,line_inv)
Iguesser_C5,a5=ferro_area(x,C5,C5,region_selection,line_inv)

#inversion using initial guesses

error_C1,inv_C1=Levenberg_Marquardt_1C(GGCD_1C,Bc1*0.8,tet1*0.8,1,1,Iguesser_C1,x,C1,constrain='N',sample_name='C1',eps=1e-15,maxiter=200)
error_C2,inv_C2=Levenberg_Marquardt_1C(GGCD_1C,Bc2*0.8,tet2*0.8,1,1,Iguesser_C2,x,C2,constrain='N',sample_name='C2',eps=1e-15,maxiter=200)
error_C3,inv_C3=Levenberg_Marquardt_1C(GGCD_1C,Bc3*0.8,tet3*0.8,1,1,Iguesser_C3,x,C3,constrain='N',sample_name='C3',eps=1e-15,maxiter=200)
error_C4,inv_C4=Levenberg_Marquardt_1C(GGCD_1C,Bc4*0.8,tet4*0.8,1,1,Iguesser_C4,x,C4,constrain='N',sample_name='C4',eps=1e-15,maxiter=200)
error_C5,inv_C5=Levenberg_Marquardt_1C(GGCD_1C,Bc5*0.9,tet5*0.8,0.5,0.5,Iguesser_C5,x,C5,constrain='N',sample_name='C5',eps=1e-15,maxiter=200)

#calculating forward models from the optimized parameters

C1_dir=GGCD_1C(inv_C1[0],inv_C1[1],inv_C1[2],inv_C1[3],inv_C1[4],x)
C2_dir=GGCD_1C(inv_C2[0],inv_C2[1],inv_C2[2],inv_C2[3],inv_C2[4],x)
C3_dir=GGCD_1C(inv_C3[0],inv_C3[1],inv_C3[2],inv_C3[3],inv_C3[4],x)
C4_dir=GGCD_1C(inv_C4[0],inv_C4[1],inv_C4[2],inv_C4[3],inv_C4[4],x)
C5_dir=GGCD_1C(inv_C5[0],inv_C5[1],inv_C5[2],inv_C5[3],inv_C5[4],x)

#calculating R²

r2_C1=R2(C1,C1_dir)
r2_C2=R2(C2,C2_dir)
r2_C3=R2(C3,C3_dir)
r2_C4=R2(C4,C4_dir)
r2_C5=R2(C5,C5_dir)

R2_sc=[r2_C1,r2_C2,r2_C3,r2_C4,r2_C5]

#plotting the data

figure,axes=plt.subplots(2,3,figsize=(15,10))
axes=axes.ravel()

axes[0].scatter(x,C1,label='$C_{1}$',color='gray',alpha=0.3)
axes[0].plot(x,C1_dir,label='Inversion',color='k')
axes[0].grid(alpha=0.5)
axes[0].legend(fontsize=12)
axes[0].set_ylabel('$\kappa$')
axes[0].set_xlabel('Applied field (T)')

axes[1].scatter(x,C2,label='$C_{2}$',color='gray',alpha=0.3)
axes[1].plot(x,C2_dir,label='Inversion',color='k')
axes[1].grid(alpha=0.5)
axes[1].legend(fontsize=12)
axes[1].set_ylabel('$\kappa$')
axes[1].set_xlabel('Applied field (T)')

axes[2].scatter(x,C3,label='$C_{3}$',color='gray',alpha=0.3)
axes[2].plot(x,C3_dir,label='Inversion',color='k')
axes[2].grid(alpha=0.5)
axes[2].legend(fontsize=12)
axes[2].set_ylabel('$\kappa$')
axes[2].set_xlabel('Applied field (T)')

axes[3].scatter(x,C4,label='$C_{4}$',color='gray',alpha=0.3)
axes[3].plot(x,C4_dir,label='Inversion',color='k')
axes[3].grid(alpha=0.5)
axes[3].legend(fontsize=12)
axes[3].set_ylabel('$\kappa$')
axes[3].set_xlabel('Applied field (T)')

axes[4].scatter(x,C5,label='$C_{5}$',color='gray',alpha=0.3)
axes[4].plot(x,C5_dir,label='Inversion',color='k')
axes[4].grid(alpha=0.5)
axes[4].legend(fontsize=12)
axes[4].set_ylabel('$\kappa$')
axes[4].set_xlabel('Applied field (T)')

axes[5].boxplot(R2_sc)
axes[5].set_ylabel('R²')

figure.tight_layout()
plt.show()

In [ ]:
#plotting table with results

fig1 = go.Figure(data=[go.Table(header=dict(values=['Parameters','$C_{1}$','$C_{2}$','$C_{3}$','$C_{4}$','$C_{5}$']),
                 cells=dict(values=[['$B_{c} \ (T)$', '$θ$','$α$','$β$','$I$'],
                                    ["{:e}".format(inv_C1[0]),"{:e}".format(inv_C1[1]),"{:e}".format(inv_C1[2]),"{:e}".format(inv_C1[3]),"{:e}".format(inv_C1[4])],
                                   ["{:e}".format(inv_C2[0]),"{:e}".format(inv_C2[1]),"{:e}".format(inv_C2[2]),"{:e}".format(inv_C2[3]),"{:e}".format(inv_C2[4])],
                                   ["{:e}".format(inv_C3[0]),"{:e}".format(inv_C3[1]),"{:e}".format(inv_C3[2]),"{:e}".format(inv_C3[3]),"{:e}".format(inv_C3[4])],
                                   ["{:e}".format(inv_C4[0]),"{:e}".format(inv_C4[1]),"{:e}".format(inv_C4[2]),"{:e}".format(inv_C4[3]),"{:e}".format(inv_C4[4])],
                                   ["{:e}".format(inv_C5[0]),"{:e}".format(inv_C5[1]),"{:e}".format(inv_C5[2]),"{:e}".format(inv_C5[3]),"{:e}".format(inv_C5[4])]],
                           ))
                     ])
fig1.update_layout(width=1000, height=400)
fig1.show()


fig3 = go.Figure(data=[go.Table(header=dict(values=['Parameters','$C_{1}$','$C_{2}$','$C_{3}$','$C_{4}$','$C_{5}$']),
                 cells=dict(values=[['$\kappa_{0}$','$||e²||_{2}$','$R²$'],
                                    ["{:e}".format(a1),"{:e}".format(error_C1),np.round(r2_C1,4)],
                                   ["{:e}".format(a2),"{:e}".format(error_C2),np.round(r2_C2,4)],
                                   ["{:e}".format(a3),"{:e}".format(error_C3),np.round(r2_C3,4)],
                                   ["{:e}".format(a4),"{:e}".format(error_C4),np.round(r2_C4,4)],
                                   ["{:e}".format(a5),"{:e}".format(error_C5),np.round(r2_C5,4)]]))])
fig3.update_layout(width=1000, height=400)
fig3.show()

<hr>
<br>
<br>
<b> Two ferromagnetic components + para/diamagnetic contribution </b>
<br>
<br>
<hr>

In [ ]:
#initial guesses for parameter I
Iguesser_C1_C2,a_C1C2=ferro_area(x,C1_C2,C1_C2,region_selection,line_inv)
Iguesser_C1_C3,a_C1C3=ferro_area(x,C1_C3,C1_C2,region_selection,line_inv)
Iguesser_C1_C4,a_C1C4=ferro_area(x,C1_C4,C1_C4,region_selection,line_inv)
Iguesser_C1_C5,a_C1C5=ferro_area(x,C1_C5,C1_C5,region_selection,line_inv)
Iguesser_C3_C5,a_C3C5=ferro_area(x,C3_C5,C3_C5,region_selection,line_inv)
Iguesser_C4_C5,a_C4C5=ferro_area(x,C4_C5,C4_C5,region_selection,line_inv)

a_C1C3=-9e-3
a_C1C4=0.4e-1
a_C1C5=-8e-3
a_C3C5=1e-5
a_C4C5=-0.5e-1

#inversion using initial guesses

error_C1_C2,inv_C1_C2=Levenberg_Marquardt_2C(GGCD_2C,Bc1,tet1,1,1,Iguesser_C1_C2,Bc2,tet2,alfa2,beta2,Iguesser_C1_C2,x,(C1_C2-a_C1C2),constrain='N',option='B',sample_name='C1_C2',eps=1e-15,maxiter=200)
error_C1_C3,inv_C1_C3=Levenberg_Marquardt_2C(GGCD_2C,Bc1*0.8,tet1*0.8,1,1,Iguesser_C1_C3,Bc3*0.8,tet3*0.8,1,1,Iguesser_C1_C3,x,(C1_C3-a_C1C3),constrain='N',option='C',sample_name='C1_C3',eps=1e-15,maxiter=200)
error_C1_C4,inv_C1_C4=Levenberg_Marquardt_2C(GGCD_2C,Bc1*0.8,tet1*0.8,1,1,Iguesser_C1_C4,Bc4*0.8,tet4*0.8,0.6,0.8,Iguesser_C1_C4,x,(C1_C4-a_C1C4),constrain='N',option='C',sample_name='C1_C4',eps=1e-15,maxiter=200)
error_C1_C5,inv_C1_C5=Levenberg_Marquardt_2C(GGCD_2C,Bc1*0.8,tet1*0.8,1,1,Iguesser_C1_C5,Bc5*0.8,tet5*0.8,0.6,0.8,Iguesser_C1_C5,x,(C1_C5-a_C1C5),constrain='N',option='C',sample_name='C1_C5',eps=1e-15,maxiter=200)
error_C3_C5,inv_C3_C5=Levenberg_Marquardt_2C(GGCD_2C,Bc3*0.8,tet3*0.8,0.5,1,Iguesser_C3_C5,Bc5*0.8,tet5*0.8,1,1,Iguesser_C3_C5,x,C3_C5-a_C3C5,constrain='N',option='C',sample_name='C1_C5',eps=1e-15,maxiter=200)
error_C4_C5,inv_C4_C5=Levenberg_Marquardt_2C(GGCD_2C,Bc4*0.8,tet4*0.8,1,1,Iguesser_C4_C5,Bc5*0.9,tet5*0.8,0.1,0.5,Iguesser_C4_C5,x,C4_C5-a_C4C5,constrain='N',option='C',sample_name='C1_C5',eps=1e-15,maxiter=200)


#calculating forward models from the optimized parameters


C1C2_dir=GGCD_2C(inv_C1_C2[0],inv_C1_C2[1],inv_C1_C2[2],inv_C1_C2[3],inv_C1_C2[4],inv_C1_C2[5],
                 inv_C1_C2[6],inv_C1_C2[7],inv_C1_C2[8],inv_C1_C2[9],x)
C1C2_dir_A=GGCD_1C(inv_C1_C2[0],inv_C1_C2[1],inv_C1_C2[2],inv_C1_C2[3],inv_C1_C2[4],x)
C1C2_dir_B=GGCD_1C(inv_C1_C2[5],inv_C1_C2[6],inv_C1_C2[7],inv_C1_C2[8],inv_C1_C2[9],x)

C1C3_dir=GGCD_2C(inv_C1_C3[0],inv_C1_C3[1],inv_C1_C3[2],inv_C1_C3[3],inv_C1_C3[4],inv_C1_C3[5],
                  inv_C1_C3[6],inv_C1_C3[7],inv_C1_C3[8],inv_C1_C3[9],x)
C1C3_dir_A=GGCD_1C(inv_C1_C3[0],inv_C1_C3[1],inv_C1_C3[2],inv_C1_C3[3],inv_C1_C3[4],x)
C1C3_dir_B=GGCD_1C(inv_C1_C3[5],inv_C1_C3[6],inv_C1_C3[7],inv_C1_C3[8],inv_C1_C3[9],x)

C1C4_dir=GGCD_2C(inv_C1_C4[0],inv_C1_C4[1],inv_C1_C4[2],inv_C1_C4[3],inv_C1_C4[4],inv_C1_C4[5],
                 inv_C1_C4[6],inv_C1_C4[7],inv_C1_C4[8],inv_C1_C4[9],x)
C1C4_dir_A=GGCD_1C(inv_C1_C4[0],inv_C1_C4[1],inv_C1_C4[2],inv_C1_C4[3],inv_C1_C4[4],x)
C1C4_dir_B=GGCD_1C(inv_C1_C4[5],inv_C1_C4[6],inv_C1_C4[7],inv_C1_C4[8],inv_C1_C4[9],x)

C1C5_dir=GGCD_2C(inv_C1_C5[0],inv_C1_C5[1],inv_C1_C5[2],inv_C1_C5[3],inv_C1_C5[4],inv_C1_C5[5],
                 inv_C1_C5[6],inv_C1_C5[7],inv_C1_C5[8],inv_C1_C5[9],x)
C1C5_dir_A=GGCD_1C(inv_C1_C5[0],inv_C1_C5[1],inv_C1_C5[2],inv_C1_C5[3],inv_C1_C5[4],x)
C1C5_dir_B=GGCD_1C(inv_C1_C5[5],inv_C1_C5[6],inv_C1_C5[7],inv_C1_C5[8],inv_C1_C5[9],x)


C3C5_dir=GGCD_2C(inv_C3_C5[0],inv_C3_C5[1],inv_C3_C5[2],inv_C3_C5[3],inv_C3_C5[4],inv_C3_C5[5],
                 inv_C3_C5[6],inv_C3_C5[7],inv_C3_C5[8],inv_C3_C5[9],x)
C3C5_dir_A=GGCD_1C(inv_C3_C5[0],inv_C3_C5[1],inv_C3_C5[2],inv_C3_C5[3],inv_C3_C5[4],x)
C3C5_dir_B=GGCD_1C(inv_C3_C5[5],inv_C3_C5[6],inv_C3_C5[7],inv_C3_C5[8],inv_C3_C5[9],x)


C4C5_dir=GGCD_2C(inv_C4_C5[0],inv_C4_C5[1],inv_C4_C5[2],inv_C4_C5[3],inv_C4_C5[4],inv_C4_C5[5],
                 inv_C4_C5[6],inv_C4_C5[7],inv_C4_C5[8],inv_C4_C5[9],x)
C4C5_dir_A=GGCD_1C(inv_C4_C5[0],inv_C4_C5[1],inv_C4_C5[2],inv_C4_C5[3],inv_C4_C5[4],x)
C4C5_dir_B=GGCD_1C(inv_C4_C5[5],inv_C4_C5[6],inv_C4_C5[7],inv_C4_C5[8],inv_C4_C5[9],x)


#calculating R²

r2_C1C2=R2(C1_C2,C1C2_dir+a_C1C2)
r2_C1C3=R2(C1_C3,C1C3_dir+a_C1C3)
r2_C1C4=R2(C1_C4,C1C4_dir+a_C1C4)
r2_C1C5=R2(C1_C5,C1C5_dir+a_C1C5)
r2_C3C5=R2(C3_C5,C3C5_dir+a_C3C5)
r2_C4C5=R2(C4_C5,C4C5_dir+a_C4C5)

#plotting the data

figure,axes=plt.subplots(3,2,figsize=(10,15))
axes=axes.ravel()

axes[0].scatter(x,C1_C2,label='Data',color='gray',alpha=0.3)
axes[0].plot(x,C1C2_dir+a_C1C2,label='$C_{a}+C_{b}+ \kappa_{0}$',color='k')
axes[0].plot(x,C1C2_dir_A,label=r'$C_{a}$',color='royalblue')
axes[0].plot(x,C1C2_dir_B,label=r'$C_{b}$',color='forestgreen')
axes[0].grid(alpha=0.5)
axes[0].legend(fontsize=11,shadow=True,loc='upper left')
axes[0].set_ylabel('$\kappa$')
axes[0].set_xlabel('Applied field (T)')
axes[0].set_title('$C_{1}+C_{2}+ \kappa_{0}$',fontsize=11)

axes[1].scatter(x,C1_C3,label='Data',color='gray',alpha=0.3)
axes[1].plot(x,C1C3_dir+a_C1C3,label='$C_{a}+C_{b}+ \kappa_{0}$',color='k')
axes[1].plot(x,C1C3_dir_A,label='$C_{a}$',color='royalblue')
axes[1].plot(x,C1C3_dir_B,label='$C_{b}$',color='forestgreen')
axes[1].grid(alpha=0.5)
axes[1].legend(fontsize=11,shadow=True,loc='upper left')
axes[1].set_ylabel('$\kappa$')
axes[1].set_xlabel('Applied field (T)')
axes[1].set_title('$C_{1}+C_{3}+ \kappa_{0}$',fontsize=11)


axes[2].scatter(x,C1_C4,label='Data',color='gray',alpha=0.3)
axes[2].plot(x,C1C4_dir+a_C1C4,label='$C_{a}+C_{b}+ \kappa_{0}$',color='k')
axes[2].plot(x,C1C4_dir_A,label='$C_{a}$',color='royalblue')
axes[2].plot(x,C1C4_dir_B,label='$C_{b}$',color='forestgreen')
axes[2].grid(alpha=0.5)
axes[2].legend(fontsize=11,shadow=True,loc='upper left')
axes[2].set_ylabel('$\kappa$')
axes[2].set_xlabel('Applied field (T)')
axes[2].set_title('$C_{1}+C_{4}+ \kappa_{0}$',fontsize=11)

axes[3].scatter(x,C1_C5,label='Data',color='gray',alpha=0.3)
axes[3].plot(x,C1C5_dir+a_C1C5,label='$C_{a}+C_{b}+ \kappa_{0}$',color='k')
axes[3].plot(x,C1C5_dir_A,label='$C_{a}$',color='royalblue')
axes[3].plot(x,C1C5_dir_B,label='$C_{b}$',color='forestgreen')
axes[3].grid(alpha=0.5)
axes[3].legend(fontsize=11,shadow=True,loc='upper left')
axes[3].set_ylabel('$\kappa$')
axes[3].set_xlabel('Applied field (T)')
axes[3].set_title('$C_{1}+C_{5}+ \kappa_{0}$',fontsize=11)

axes[4].scatter(x,C3_C5,label='Data',color='gray',alpha=0.3)
axes[4].plot(x,C3C5_dir+a_C3C5,label='$C_{a}+C_{b}+ \kappa_{0}$',color='k')
axes[4].plot(x,C3C5_dir_A,label='$C_{a}$',color='royalblue')
axes[4].plot(x,C3C5_dir_B,label='$C_{b}$',color='forestgreen')
axes[4].grid(alpha=0.5)
axes[4].legend(fontsize=11,shadow=True,loc='upper left')
axes[4].set_ylabel('$\kappa$')
axes[4].set_xlabel('Applied field (T)')
axes[4].set_title('$C_{3}+C_{5}+ \kappa_{0}$',fontsize=11)

axes[5].scatter(x,C4_C5,label='Data',color='gray',alpha=0.3)
axes[5].plot(x,C4C5_dir+a_C4C5,label='$C_{a}+C_{b}+ \kappa_{0}$',color='k')
axes[5].plot(x,C4C5_dir_A,label='$C_{a}$',color='royalblue')
axes[5].plot(x,C4C5_dir_B,label='$C_{b}$',color='forestgreen')
axes[5].grid(alpha=0.5)
axes[5].legend(fontsize=11,shadow=True,loc='upper left')
axes[5].set_ylabel('$\kappa$')
axes[5].set_xlabel('Applied field (T)')
axes[5].set_title('$C_{4}+C_{5}+ \kappa_{0}$',fontsize=11)



Afont = {'family': 'serif',
        'backgroundcolor': 'white',
        'weight': 'normal',
        'size':10,
        }

at1 = AnchoredText(
    "a", prop=Afont, frameon=True, loc='upper right')
at1.patch.set_boxstyle("square,pad=0.")
axes[0].add_artist(at1)

at2 = AnchoredText(
    "b", prop=Afont, frameon=True, loc='upper right')
at2.patch.set_boxstyle("square,pad=0.")
axes[1].add_artist(at2)

at3 = AnchoredText(
    "c", prop=Afont, frameon=True, loc='upper right')
at3.patch.set_boxstyle("square,pad=0.")
axes[2].add_artist(at3)

at4 = AnchoredText(
    "d", prop=Afont, frameon=True, loc='upper right')
at4.patch.set_boxstyle("square,pad=0.")
axes[3].add_artist(at4)

at5 = AnchoredText(
    "e", prop=Afont, frameon=True, loc='upper right')
at5.patch.set_boxstyle("square,pad=0.")
axes[4].add_artist(at5)

at6 = AnchoredText(
    "f", prop=Afont, frameon=True, loc='upper right')
at6.patch.set_boxstyle("square,pad=0.")
axes[5].add_artist(at6)

figure.tight_layout()


In [ ]:
#plotting table with results

fig1 = go.Figure(data=[go.Table(header=dict(values=['Parameters','$C_{1}+C_{2}$','$C_{1}+C_{3}$','$C_{1}+C_{4}$','$C_{1}+C_{5}$','$C_{3}+C_{5}$','$C_{4}+C_{5}$']),
                 cells=dict(values=[['$B_{c1} \ (T)$', '$θ_{1}$','$α_{1}$','$β_{1}$','$I_{1}$','$B_{c2} \ (T)$', '$θ_{2}$','$α_{2}$','$β_{2}$','$I_{2}$'],
                                    ["{:e}".format(inv_C1_C2[0]),"{:e}".format(inv_C1_C2[1]),"{:e}".format(inv_C1_C2[2]),"{:e}".format(inv_C1_C2[3]),"{:e}".format(inv_C1_C2[4]),
                                    "{:e}".format(inv_C1_C2[5]),"{:e}".format(inv_C1_C2[6]),"{:e}".format(inv_C1_C2[7]),"{:e}".format(inv_C1_C2[8]),"{:e}".format(inv_C1_C2[9])],
                                    ["{:e}".format(inv_C1_C3[0]),"{:e}".format(inv_C1_C3[1]),"{:e}".format(inv_C1_C3[2]),"{:e}".format(inv_C1_C3[3]),"{:e}".format(inv_C1_C3[4]),
                                    "{:e}".format(inv_C1_C3[5]),"{:e}".format(inv_C1_C3[6]),"{:e}".format(inv_C1_C3[7]),"{:e}".format(inv_C1_C3[8]),"{:e}".format(inv_C1_C3[9])],
                                   ["{:e}".format(inv_C1_C4[0]),"{:e}".format(inv_C1_C4[1]),"{:e}".format(inv_C1_C4[2]),"{:e}".format(inv_C1_C4[3]),"{:e}".format(inv_C1_C4[4]),
                                    "{:e}".format(inv_C1_C4[5]),"{:e}".format(inv_C1_C4[6]),"{:e}".format(inv_C1_C4[7]),"{:e}".format(inv_C1_C4[8]),"{:e}".format(inv_C1_C4[9])],
                                   ["{:e}".format(inv_C1_C5[0]),"{:e}".format(inv_C1_C5[1]),"{:e}".format(inv_C1_C5[2]),"{:e}".format(inv_C1_C5[3]),"{:e}".format(inv_C1_C5[4]),
                                    "{:e}".format(inv_C1_C5[5]),"{:e}".format(inv_C1_C5[6]),"{:e}".format(inv_C1_C5[7]),"{:e}".format(inv_C1_C5[8]),"{:e}".format(inv_C1_C5[9])],
                                   ["{:e}".format(inv_C3_C5[0]),"{:e}".format(inv_C3_C5[1]),"{:e}".format(inv_C3_C5[2]),"{:e}".format(inv_C3_C5[3]),"{:e}".format(inv_C3_C5[4]),
                                    "{:e}".format(inv_C3_C5[5]),"{:e}".format(inv_C3_C5[6]),"{:e}".format(inv_C3_C5[7]),"{:e}".format(inv_C3_C5[8]),"{:e}".format(inv_C3_C5[9])],
                                   ["{:e}".format(inv_C4_C5[0]),"{:e}".format(inv_C4_C5[1]),"{:e}".format(inv_C4_C5[2]),"{:e}".format(inv_C4_C5[3]),"{:e}".format(inv_C4_C5[4]),
                                    "{:e}".format(inv_C4_C5[5]),"{:e}".format(inv_C4_C5[6]),"{:e}".format(inv_C4_C5[7]),"{:e}".format(inv_C4_C5[8]),"{:e}".format(inv_C4_C5[9])]],
                           ))
                     ])
fig1.update_layout(width=1000, height=400)
fig1.show()


fig3 = go.Figure(data=[go.Table(header=dict(values=['Parameters','$C_{1}+C_{2}$','$C_{1}+C_{3}$','$C_{1}+C_{4}$','$C_{1}+C_{5}$','$C_{3}+C_{5}$','$C_{4}+C_{5}$']),
                 cells=dict(values=[['$\kappa_{0}$','$||e²||_{2}$','$R²$'],
                                    ["{:e}".format(a_C1C2),"{:e}".format(error_C1_C2),np.round(r2_C1C2,4)],
                                    ["{:e}".format(a_C1C3),"{:e}".format(error_C1_C3),np.round(r2_C1C3,4)],
                                    ["{:e}".format(a_C1C4),"{:e}".format(error_C1_C4),np.round(r2_C1C4,4)],
                                    ["{:e}".format(a_C1C5),"{:e}".format(error_C1_C5),np.round(r2_C1C5,4)],
                                    ["{:e}".format(a_C3C5),"{:e}".format(error_C3_C5),np.round(r2_C3C5,4)],
                                    ["{:e}".format(a_C4C5),"{:e}".format(error_C4_C5),np.round(r2_C4C5,4)]
                                   ]))])
fig3.update_layout(width=1000, height=400)
fig3.show()

<hr>
<br>
<br>
<b> Testing noise and sampling on two ferromagnetic components + para/diamagnetic contribution </b>
<br>
<br>
<hr>

In [ ]:
#testing new noise values

noise2=1e-5
noise3=5e-5
N2=500
N3=200
x1=np.linspace(-1,1,N2)
x2=np.linspace(-1,1,N3)


H1_n2=CDF(Bc1,tet1,alfa1,beta1,i1,x1)
H2_n2=CDF(Bc2,tet2,alfa2,beta2,i2,x1)
H3_n2=CDF(Bc3,tet3,alfa3,beta3,i3,x1)

H1_n3=CDF(Bc1,tet1,alfa1,beta1,i1,x2)
H2_n3=CDF(Bc2,tet2,alfa2,beta2,i2,x2)
H3_n3=CDF(Bc3,tet3,alfa3,beta3,i3,x2)

#generating new synthetic data

C1_C3_noise2=np.gradient((H1_nl+H3_nl+np.random.normal(0,noise2,np.size(x))),x)+X0_3
C1_C3_noise3=np.gradient((H1_nl+H3_nl+np.random.normal(0,noise3,np.size(x))),x)+X0_3

C1_C3_N2=np.gradient((H1_n2+H3_n2+np.random.normal(0,noise3,np.size(x1))),x1)+X0_3
C1_C3_N3=np.gradient((H1_n3+H3_n3+np.random.normal(0,noise3,np.size(x2))),x2)+X0_3

C1_C2_noise2=np.gradient((H1_nl+H2_nl+np.random.normal(0,noise2,np.size(x))),x)+X0_1
C1_C2_noise3=np.gradient((H1_nl+H2_nl+np.random.normal(0,noise3,np.size(x))),x)+X0_1

C1_C2_N2=np.gradient((H1_n2+H2_n2+np.random.normal(0,noise3,np.size(x1))),x1)+X0_1
C1_C2_N3=np.gradient((H1_n3+H2_n3+np.random.normal(0,noise3,np.size(x2))),x2)+X0_1

# inversion procedure

error_C1_C3_noise2,inv_C1_C3_noise2=Levenberg_Marquardt_2C(GGCD_2C,Bc1*0.8,tet1*0.8,1,1,Iguesser_C1_C3,Bc3*0.8,tet3*0.8,1,0.8,Iguesser_C1_C3,x,(C1_C3_noise2-a_C1C3),constrain='N',option='C',sample_name='C1_C3',eps=1e-15,maxiter=200)
error_C1_C3_noise3,inv_C1_C3_noise3=Levenberg_Marquardt_2C(GGCD_2C,Bc1*0.8,tet1*0.8,1,1,Iguesser_C1_C3,Bc3*0.8,tet3*0.8,1,0.8,Iguesser_C1_C3,x,(C1_C3_noise3-a_C1C3),constrain='N',option='C',sample_name='C1_C3',eps=1e-15,maxiter=200)
error_C1_C3_N2,inv_C1_C3_N2=Levenberg_Marquardt_2C(GGCD_2C,Bc1*0.8,tet1*0.8,1,1,Iguesser_C1_C3,Bc3*0.8,tet3*0.8,1,0.5,Iguesser_C1_C3,x1,(C1_C3_N2-a_C1C3),constrain='N',option='C',sample_name='C1_C3',eps=1e-15,maxiter=200)
error_C1_C3_N3,inv_C1_C3_N3=Levenberg_Marquardt_2C(GGCD_2C,Bc1*0.8,tet1*0.8,1,1,Iguesser_C1_C3,Bc3*0.8,tet3*0.8,1,0.5,Iguesser_C1_C3,x2,(C1_C3_N3-a_C1C3),constrain='N',option='C',sample_name='C1_C3',eps=1e-15,maxiter=200)



error_C1_C2_noise2,inv_C1_C2_noise2=Levenberg_Marquardt_2C(GGCD_2C,Bc1*0.8,tet1*0.8,1,1,Iguesser_C1_C2,Bc2*0.8,tet2*0.8,1,1,Iguesser_C1_C2,x,(C1_C2_noise2-a_C1C2),constrain='N',option='C',sample_name='C1_C2',eps=1e-15,maxiter=200)
error_C1_C2_noise3,inv_C1_C2_noise3=Levenberg_Marquardt_2C(GGCD_2C,Bc1*0.8,tet1*0.8,1,1,Iguesser_C1_C2,Bc2*0.8,tet2*0.8,1,1,Iguesser_C1_C2,x,(C1_C2_noise3-a_C1C2),constrain='N',option='C',sample_name='C1_C2',eps=1e-15,maxiter=200)
error_C1_C2_N2,inv_C1_C2_N2=Levenberg_Marquardt_2C(GGCD_2C,Bc1*0.8,tet1*0.8,1,1,Iguesser_C1_C2,Bc2*0.8,tet2*0.8,1,1,Iguesser_C1_C2,x1,(C1_C2_N2-a_C1C2),constrain='N',option='C',sample_name='C1_C2',eps=1e-15,maxiter=200)
error_C1_C2_N3,inv_C1_C2_N3=Levenberg_Marquardt_2C(GGCD_2C,Bc1*0.8,tet1*0.8,1,1,Iguesser_C1_C2,Bc2*0.8,tet2*0.8,1,1,Iguesser_C1_C2,x2,(C1_C2_N3-a_C1C2),constrain='N',option='C',sample_name='C1_C2',eps=1e-15,maxiter=200)

#calculating forward models from the optimized parameters

C1C2_dir_noise2=GGCD_2C(inv_C1_C2_noise2[0],inv_C1_C2_noise2[1],inv_C1_C2_noise2[2],inv_C1_C2_noise2[3],inv_C1_C2_noise2[4],inv_C1_C2_noise2[5],
                 inv_C1_C2_noise2[6],inv_C1_C2_noise2[7],inv_C1_C2_noise2[8],inv_C1_C2_noise2[9],x)
C1C2_dir_noise3=GGCD_2C(inv_C1_C2_noise3[0],inv_C1_C2_noise3[1],inv_C1_C2_noise3[2],inv_C1_C2_noise3[3],inv_C1_C2_noise3[4],inv_C1_C2_noise3[5],
                 inv_C1_C2_noise3[6],inv_C1_C2_noise3[7],inv_C1_C2_noise3[8],inv_C1_C2_noise3[9],x)
C1C2_dir_N2=GGCD_2C(inv_C1_C2_N2[0],inv_C1_C2_N2[1],inv_C1_C2_N2[2],inv_C1_C2_N2[3],inv_C1_C2_N2[4],inv_C1_C2_N2[5],
                 inv_C1_C2_N2[6],inv_C1_C2_N2[7],inv_C1_C2_N2[8],inv_C1_C2_N2[9],x1)
C1C2_dir_N3=GGCD_2C(inv_C1_C2_N3[0],inv_C1_C2_N3[1],inv_C1_C2_N3[2],inv_C1_C2_N3[3],inv_C1_C2_N3[4],inv_C1_C2_N3[5],
                 inv_C1_C2_N3[6],inv_C1_C2_N3[7],inv_C1_C2_N3[8],inv_C1_C2_N3[9],x2)


C1C3_dir_noise2=GGCD_2C(inv_C1_C3_noise2[0],inv_C1_C3_noise2[1],inv_C1_C3_noise2[2],inv_C1_C3_noise2[3],inv_C1_C3_noise2[4],inv_C1_C3_noise2[5],
                 inv_C1_C3_noise2[6],inv_C1_C3_noise2[7],inv_C1_C3_noise2[8],inv_C1_C3_noise2[9],x)
C1C3_dir_noise3=GGCD_2C(inv_C1_C3_noise3[0],inv_C1_C3_noise3[1],inv_C1_C3_noise3[2],inv_C1_C3_noise3[3],inv_C1_C3_noise3[4],inv_C1_C3_noise3[5],
                 inv_C1_C3_noise3[6],inv_C1_C3_noise3[7],inv_C1_C3_noise3[8],inv_C1_C3_noise3[9],x)
C1C3_dir_N2=GGCD_2C(inv_C1_C3_N2[0],inv_C1_C3_N2[1],inv_C1_C3_N2[2],inv_C1_C3_N2[3],inv_C1_C3_N2[4],inv_C1_C3_N2[5],
                 inv_C1_C3_N2[6],inv_C1_C3_N2[7],inv_C1_C3_N2[8],inv_C1_C3_N2[9],x1)
C1C3_dir_N3=GGCD_2C(inv_C1_C3_N3[0],inv_C1_C3_N3[1],inv_C1_C3_N3[2],inv_C1_C3_N3[3],inv_C1_C3_N3[4],inv_C1_C3_N3[5],
                 inv_C1_C3_N3[6],inv_C1_C3_N3[7],inv_C1_C3_N3[8],inv_C1_C3_N3[9],x2)

#calculating R²

r2_C1C2_noise2=R2(C1_C2_noise2,C1C2_dir_noise2+a_C1C2)
r2_C1C2_noise3=R2(C1_C2_noise3,C1C2_dir_noise3+a_C1C2)
r2_C1C2_N2=R2(C1_C2_N2,C1C2_dir_N2+a_C1C2)
r2_C1C2_N3=R2(C1_C2_N3,C1C2_dir_N3+a_C1C2)


r2_C1C3_noise2=R2(C1_C3_noise2,C1C3_dir_noise2+a_C1C3)
r2_C1C3_noise3=R2(C1_C3_noise3,C1C3_dir_noise3+a_C1C3)
r2_C1C3_N2=R2(C1_C3_N2,C1C3_dir_N2+a_C1C3)
r2_C1C3_N3=R2(C1_C3_N3,C1C3_dir_N3+a_C1C3)

#plotting the data

string1='c'

figure,axes=plt.subplots(2,2,figsize=(10,10))

axes=axes.ravel()

axes[0].scatter(x,C1_C3_noise2,marker='.',color='green',alpha=0.4,zorder=20,label=r"$A:\ \eta={0:s},N=1000$".format(as_si(noise2,2)))
axes[0].scatter(x,C1_C3_noise3,marker='.',color='blue',alpha=0.3,label=r"$B:\ \eta={0:s},N=1000$".format(as_si(noise3,2)))
axes[0].plot(x,C1C3_dir_noise2+a_C1C3,zorder=30,color='r',label=f'$A_{(string1)}:\ R^{2}={np.round(r2_C1C3_noise2,4)},||e^{2}||={np.round(error_C1_C3_noise2,4)}$',linewidth=2.0)
axes[0].plot(x,C1C3_dir_noise3+a_C1C3,zorder=30,color='yellow',label=f'$B_{(string1)}:\ R^{2}={np.round(r2_C1C3_noise3,4)},||e^{2}||={np.round(error_C1_C3_noise3,4)}$',linewidth=2.0)
axes[0].grid(alpha=0.5)
axes[0].legend(fontsize=10,shadow=True,loc='upper left')
axes[0].set_ylabel('$\kappa$')
axes[0].set_xlabel('Applied field (T)')
axes[0].set_ylim((min(C1_C3_noise3)+(min(C1_C3_noise3)*0.1),(max(C1_C3_noise3)+max(C1_C3_noise3)*0.33)))
axes[0].set_title('$C_{1}+C_{3}+\kappa_{0}$',fontsize=10)
axes[0].set_facecolor('whitesmoke')


axes[1].scatter(x1,C1_C3_N2,marker='.',color='green',alpha=0.5,label=r"$A:\ \eta={0:s},N=500$".format(as_si(noise3,2)))
axes[1].scatter(x2,C1_C3_N3,marker='.',color='blue',alpha=0.7,label=r"$B:\ \eta={0:s},N=200$".format(as_si(noise3,2)),zorder=20)
axes[1].plot(x1,C1C3_dir_N2+a_C1C3,zorder=30,color='r',label=f'$A_{(string1)}:\ R^{2}={np.round(r2_C1C3_N3,4)},||e^{2}||={np.round(error_C1_C3_N3,4)}$',linewidth=2.0)
axes[1].plot(x2,C1C3_dir_N3+a_C1C3,zorder=30,color='yellow',label=f'$B_{(string1)}:\ R^{2}={np.round(r2_C1C3_N2,4)},||e^{2}||={np.round(error_C1_C3_N2,4)}$',linewidth=2.0)
axes[1].grid(alpha=0.5)
axes[1].legend(fontsize=10,shadow=True,loc='upper left')
axes[1].set_ylabel('$\kappa$')
axes[1].set_xlabel('Applied field (T)')
axes[1].set_ylim((min(C1_C3_N2)+(min(C1_C3_N2)*0.5),(max(C1_C3_N2)+max(C1_C3_N2)*0.33)))
axes[1].set_title('$C_{1}+C_{3}+\kappa_{0}$',fontsize=10)
axes[1].set_facecolor('whitesmoke')

axes[2].scatter(x,C1_C2_noise2,marker='.',color='green',alpha=0.4,zorder=20,label=r"$A:\ \eta={0:s},N=1000$".format(as_si(noise2,2)))
axes[2].scatter(x,C1_C2_noise3,marker='.',color='blue',alpha=0.3,label=r"$B:\ \eta={0:s},N=1000$".format(as_si(noise3,2)))
axes[2].plot(x,C1C2_dir_noise2+a_C1C2,zorder=30,color='r',label=f'$A_{(string1)}:\ R^{2}={np.round(r2_C1C2_noise2,4)},||e^{2}||={np.round(error_C1_C2_noise2,4)}$',linewidth=2.0)
axes[2].plot(x,C1C2_dir_noise3+a_C1C2,zorder=30,color='yellow',label=f'$B_{(string1)}:\ R^{2}={np.round(r2_C1C2_noise3,4)},||e^{2}||={np.round(error_C1_C2_noise3,4)}$',linewidth=2.0)
axes[2].grid(alpha=0.5)
axes[2].legend(fontsize=10,shadow=True,loc='upper left')
axes[2].set_ylabel('$\kappa$')
axes[2].set_xlabel('Applied field (T)')
axes[2].set_ylim((min(C1_C2_noise3)+(min(C1_C2_noise3)*0.1),(max(C1_C2_noise3)+max(C1_C2_noise3)*0.33)))
axes[2].set_title('$C_{1}+C_{2}+\kappa_{0}$',fontsize=10)
axes[2].set_facecolor('whitesmoke')

axes[3].scatter(x1,C1_C2_N2,marker='.',color='green',alpha=0.5,label=r"$A:\ \eta={0:s},N=500$".format(as_si(noise3,2)))
axes[3].scatter(x2,C1_C2_N3,marker='.',color='blue',alpha=0.7,label=r"$B:\ \eta={0:s},N=200$".format(as_si(noise3,2)),zorder=20)
axes[3].plot(x1,C1C2_dir_N2+a_C1C2,zorder=30,color='r',label=f'$A_{(string1)}:\ R^{2}={np.round(r2_C1C2_N3,4)},||e^{2}||={np.round(error_C1_C2_N3,4)}$',linewidth=2.0)
axes[3].plot(x2,C1C2_dir_N3+a_C1C2,zorder=30,color='yellow',label=f'$B_{(string1)}:\ R^{2}={np.round(r2_C1C2_N2,4)},||e^{2}||={np.round(error_C1_C2_N2,4)}$',linewidth=2.0)
axes[3].grid(alpha=0.5)
axes[3].legend(fontsize=10,shadow=True,loc='upper left')
axes[3].set_ylabel('$\kappa$')
axes[3].set_xlabel('Applied field (T)')
axes[3].set_ylim((min(C1_C2_N2)+(min(C1_C2_N2)*0.5),(max(C1_C2_N2)+max(C1_C2_N2)*0.33)))
axes[3].set_title('$C_{1}+C_{2}+\kappa_{0}$',fontsize=9.5)
axes[3].set_facecolor('whitesmoke')


Afont = {'family': 'serif',
        'backgroundcolor': 'whitesmoke',
        'weight': 'normal',
        'size':10,
        }

at1 = AnchoredText(
    "a", prop=Afont, frameon=True, loc='upper right')
at1.patch.set_boxstyle("square,pad=0.")
axes[0].add_artist(at1)

at2 = AnchoredText(
    "b", prop=Afont, frameon=True, loc='upper right')
at2.patch.set_boxstyle("square,pad=0.")
axes[1].add_artist(at2)

at3 = AnchoredText(
    "c", prop=Afont, frameon=True, loc='upper right')
at3.patch.set_boxstyle("square,pad=0.")
axes[2].add_artist(at3)

at4 = AnchoredText(
    "d", prop=Afont, frameon=True, loc='upper right')
at4.patch.set_boxstyle("square,pad=0.")
axes[3].add_artist(at4)

figure.tight_layout()

<hr>
<br>
<br>
<b> Three ferromagnetic components + para/diamagnetic contribution </b>
<br>
<br>
<hr>

In [ ]:
#initial guesses for parameter I

Iguesser_C1_C3_C5,a_C1_C3_C5=ferro_area(x,C1_C3_C5,C1_C3_C5,region_selection,line_inv)
Iguesser_C3_C4_C5,a_C3_C4_C5=ferro_area(x,C3_C4_C5,C3_C4_C5,region_selection,line_inv)


# inversion procedure

error_C1_C3_C5,inv_C1_C3_C5=Levenberg_Marquardt_3C(GGCD_3C,Bc1*0.8,tet1*0.8,1,1,Iguesser_C1_C3_C5,Bc3*0.8,tet3*0.8,1,1,Iguesser_C1_C3_C5,
                                                   Bc5*0.8,tet5*0.8,1,1,Iguesser_C1_C3_C5,x,(C1_C3_C5-a_C1_C3_C5),constrain='N',option='C',sample_name='C1_C2',eps=1e-15,maxiter=200)


error_C3_C4_C5,inv_C3_C4_C5=Levenberg_Marquardt_3C(GGCD_3C,Bc3*0.8,tet3*0.8,1,1,Iguesser_C1_C3_C5,Bc4*0.8,tet4,1,1,Iguesser_C3_C4_C5,
                                                   Bc5*0.8,tet5,1,1,Iguesser_C3_C4_C5,x,(C3_C4_C5-a_C3_C4_C5),constrain='Y',option='C',sample_name='C3_C4_C5',eps=1e-15,maxiter=200)

#calculating forward models from the optimized parameters

C1_C3_C5_dir=GGCD_3C(inv_C1_C3_C5[0],inv_C1_C3_C5[1],inv_C1_C3_C5[2],inv_C1_C3_C5[3],inv_C1_C3_C5[4],inv_C1_C3_C5[5],
                 inv_C1_C3_C5[6],inv_C1_C3_C5[7],inv_C1_C3_C5[8],inv_C1_C3_C5[9],inv_C1_C3_C5[10],
                     inv_C1_C3_C5[11],inv_C1_C3_C5[12],inv_C1_C3_C5[13],inv_C1_C3_C5[14],x)
C1_C3_C5_dir_A=GGCD_1C(inv_C1_C3_C5[0],inv_C1_C3_C5[1],inv_C1_C3_C5[2],inv_C1_C3_C5[3],inv_C1_C3_C5[4],x)
C1_C3_C5_dir_B=GGCD_1C(inv_C1_C3_C5[5],inv_C1_C3_C5[6],inv_C1_C3_C5[7],inv_C1_C3_C5[8],inv_C1_C3_C5[9],x)
C1_C3_C5_dir_C=GGCD_1C(inv_C1_C3_C5[10],inv_C1_C3_C5[11],inv_C1_C3_C5[12],inv_C1_C3_C5[13],inv_C1_C3_C5[14],x)

C3_C4_C5_dir=GGCD_3C(inv_C3_C4_C5[0],inv_C3_C4_C5[1],inv_C3_C4_C5[2],inv_C3_C4_C5[3],inv_C3_C4_C5[4],inv_C3_C4_C5[5],
                 inv_C3_C4_C5[6],inv_C3_C4_C5[7],inv_C3_C4_C5[8],inv_C3_C4_C5[9],inv_C3_C4_C5[10],
                     inv_C3_C4_C5[11],inv_C3_C4_C5[12],inv_C3_C4_C5[13],inv_C3_C4_C5[14],x)

C3_C4_C5_dir_A=GGCD_1C(inv_C3_C4_C5[0],inv_C3_C4_C5[1],inv_C3_C4_C5[2],inv_C3_C4_C5[3],inv_C3_C4_C5[4],x)
C3_C4_C5_dir_B=GGCD_1C(inv_C3_C4_C5[5],inv_C3_C4_C5[6],inv_C3_C4_C5[7],inv_C3_C4_C5[8],inv_C3_C4_C5[9],x)
C3_C4_C5_dir_C=GGCD_1C(inv_C3_C4_C5[10],inv_C3_C4_C5[11],inv_C3_C4_C5[12],inv_C3_C4_C5[13],inv_C3_C4_C5[14],x)





#-----------------------------------------------------------------------------------------------------------
# LET US CONSIDER TWO COMPONENTS ONLY FOR THE SECOND CASE

# inversion procedure

error_case2,inv_case2=Levenberg_Marquardt_2C(GGCD_2C,Bc3*0.8,tet3*0.8,1,1,Iguesser_C1_C3_C5,Bc4*0.8,tet4,1,1,Iguesser_C3_C4_C5,x,(C3_C4_C5-a_C3_C4_C5),constrain='N',option='C',sample_name='C1_C2',eps=1e-15,maxiter=200)

#calculating forward models from the optimized parameters

case2_dir=GGCD_2C(inv_case2[0],inv_case2[1],inv_case2[2],inv_case2[3],inv_case2[4],inv_case2[5],
                 inv_case2[6],inv_case2[7],inv_case2[8],inv_case2[9],x)

case2_dir_A=GGCD_1C(inv_case2[0],inv_case2[1],inv_case2[2],inv_case2[3],inv_case2[4],x)
case2_dir_B=GGCD_1C(inv_case2[5],inv_case2[6],inv_case2[7],inv_case2[8],inv_case2[9],x)

In [ ]:
# calculating a grid with distinct parameters for the objetive functions

xa,xb=np.linspace(0,1,100),np.linspace(0,1,100)

objetive_f1=[]
objetive_f2=[]

for i in range(np.size(xa)):
    for j in range(np.size(xb)):
        predicted_model1=GGCD_3C(inv_C3_C4_C5[0],inv_C3_C4_C5[1],inv_C3_C4_C5[2],inv_C3_C4_C5[3],
                                inv_C3_C4_C5[4],xa[i],inv_C3_C4_C5[6],inv_C3_C4_C5[7],
                                inv_C3_C4_C5[8],inv_C3_C4_C5[9],xb[j],inv_C3_C4_C5[11],
                                inv_C3_C4_C5[12],inv_C3_C4_C5[13],inv_C3_C4_C5[14],x)
        predicted_model2=GGCD_2C(xa[i],inv_case2[1],inv_case2[2],inv_case2[3],
                                inv_case2[4],xb[j],inv_case2[6],inv_case2[7],inv_case2[8],inv_case2[9],x)       
        objetive_f1=np.append(objetive_f1,np.linalg.norm((C3_C4_C5-(predicted_model1+a_C3_C4_C5))**2))
        objetive_f2=np.append(objetive_f2,np.linalg.norm((C3_C4_C5-(predicted_model2+a_C3_C4_C5))**2))
        
Xa,Xb=np.meshgrid(xa,xb)
objective_fr1=np.reshape(objetive_f1,np.shape(Xa))
objective_fr2=np.reshape(objetive_f2,np.shape(Xa))

#updating some plotting parameters

plt.rcParams.update({"text.usetex": True, "font.family": "serif", "font.size": 13})

#calculating R²

r2_C1_C3_C5=R2(C1_C3_C5,C1_C3_C5_dir+a_C1_C3_C5)
r2_C3_C4_C5=R2(C3_C4_C5,C3_C4_C5_dir+a_C3_C4_C5)
r2_case2=R2(C3_C4_C5,case2_dir+a_C3_C4_C5)

#plotting the data

figure,ax=plt.subplots(figsize=(10,8))

gs = gridspec.GridSpec(4, 2)
ax1 = plt.subplot(gs[0:2,:-1])
ax2 = plt.subplot(gs[0, 1])
ax3= plt.subplot(gs[1, 1])
ax4 = plt.subplot(gs[2:4, 0])
ax5 = plt.subplot(gs[2:4, 1])


ax1.scatter(x,C1_C3_C5,marker='.',color='gray',alpha=0.5)
ax1.plot(x,C1_C3_C5_dir+a_C1_C3_C5,label='$C_{a}+C_{b}+C_{c}+ \kappa_{0}$',color='k')
ax1.plot(x,C1_C3_C5_dir_A,label='$C_{a}$',color='royalblue')
ax1.plot(x,C1_C3_C5_dir_B,label='$C_{b}$',color='forestgreen')
ax1.plot(x,C1_C3_C5_dir_C,label='$C_{c}$',color='brown')
ax1.grid(alpha=0.5)
ax1.legend(fontsize=10,shadow=False,loc='upper left')
ax1.set_ylabel('$\kappa$')
ax1.set_xlabel('Applied field (T)')
ax1.set_ylim((min(C1_C3_C5)-(min(C1_C3_C5)*2),(max(C1_C3_C5)+max(C1_C3_C5)*0.1)))
ax1.set_title(f'$C_{1}+C_{3}+C_{5}+ \kappa_{0},R^{2}={np.round(r2_C1_C3_C5,4)}$',fontsize=12)


ax2.scatter(x,C3_C4_C5,marker='.',color='gray',alpha=0.5)
ax2.plot(x,C3_C4_C5_dir+a_C3_C4_C5,label='$C_{a}+C_{b}+C_{c}+ \kappa_{0}$',color='k')
ax2.plot(x,C3_C4_C5_dir_A,label='$C_{a}$',color='royalblue')
ax2.plot(x,C3_C4_C5_dir_B,label='$C_{b}$',color='forestgreen')
ax2.plot(x,C3_C4_C5_dir_C,label='$C_{c}$',color='brown')
ax2.grid(alpha=0.5)
ax2.set_ylabel('$\kappa$')
ax2.set_xlabel('Applied field (T)')
ax2.set_ylim((min(C3_C4_C5)-(min(C3_C4_C5)*1.2),(max(C3_C4_C5)+max(C3_C4_C5)*0.33)))
ax2.set_title(f'$C_{3}+C_{4}+C_{5}+ \kappa_{0},R^{2}={np.round(r2_C3_C4_C5,4)}$',fontsize=12)


ax3.scatter(x,C3_C4_C5,marker='.',color='gray',alpha=0.5)
ax3.plot(x,case2_dir+a_C3_C4_C5,label='$C_{a}+C_{b}+ \kappa_{0}$',color='k')
ax3.plot(x,case2_dir_A,label='$C_{a}$',color='royalblue')
ax3.plot(x,case2_dir_B,label='$C_{b}$',color='forestgreen')
ax3.grid(alpha=0.5)
ax3.set_ylabel('$\kappa$')
ax3.set_xlabel('Applied field (T)')
ax3.set_ylim((min(C3_C4_C5)-(min(C3_C4_C5)*1.2),(max(C3_C4_C5)+max(C3_C4_C5)*0.33)))
ax3.set_title(f'$C_{3}+C_{4}+C_{5}+ \kappa_{0},R^{2}={np.round(r2_case2,4)}$',fontsize=12)


surf=ax4.contourf(Xa, Xb,np.log10(objective_fr1),levels=50, cmap='viridis')
surf2=ax4.contour(Xa, Xb,np.log10(objective_fr1),colors='k',levels=20,linewidths=0.5)
ax4.scatter(inv_C3_C4_C5[10],inv_C3_C4_C5[5],marker='s',color='white',edgecolor='k',zorder=70,s=70,label='Inversion')
ax4.set_xlabel('$Bc_{b}$')
ax4.set_ylabel('$Bc_{c}$')
ax4.legend(fontsize=10,shadow=False,loc='upper left')
cbar = plt.colorbar(surf,ax=ax4,orientation='horizontal',pad=0.17,shrink=0.6,ticks=[-3.6,-2.6,-1.6,-0.6]).set_label(label=r'$log_{10}||e^{2}||$',
                                                                                  labelpad=7,fontsize=11)

surf3=ax5.contourf(Xa, Xb,np.log10(objective_fr2),levels=50, cmap='viridis')
surf4=ax5.contour(Xa, Xb,np.log10(objective_fr2),colors='k',levels=20,linewidths=0.5)
ax5.scatter(inv_case2[5],inv_case2[0],marker='s',color='white',edgecolor='k',zorder=20,s=70,label='Inversion')
ax5.legend(fontsize=11,shadow=False,loc='upper left')
ax5.set_xlabel('$Bc_{b}$')
ax5.set_ylabel('$Bc_{a}$')
cbar = plt.colorbar(surf,ax=ax5,orientation='horizontal',pad=0.17,shrink=0.6,ticks=[-3.6,-2.6,-1.6,-0.6]).set_label(label=r'$log_{10}||e^{2}||$',
                                                                                  labelpad=7,fontsize=11)


figure.tight_layout()


Afont = {'family': 'serif',
        'backgroundcolor': 'white',
        'weight': 'normal',
        'size':11,
        }

at1 = AnchoredText(
    "a", prop=Afont, frameon=True, loc='upper right')
at1.patch.set_boxstyle("square,pad=0.")
ax1.add_artist(at1)

at2 = AnchoredText(
    "b", prop=Afont, frameon=True, loc='upper right')
at2.patch.set_boxstyle("square,pad=0.")
ax2.add_artist(at2)

at3 = AnchoredText(
    "c", prop=Afont, frameon=True, loc='upper right')
at3.patch.set_boxstyle("square,pad=0.")
ax3.add_artist(at3)

at4 = AnchoredText(
    "b'", prop=Afont, frameon=True, loc='upper right')
at4.patch.set_boxstyle("square,pad=0.")
ax4.add_artist(at4)

at5 = AnchoredText(
    "c'", prop=Afont, frameon=True, loc='upper right')
at5.patch.set_boxstyle("square,pad=0.")
ax5.add_artist(at5)

<br>
<br>

<hr>
<br>
<br>
<b> Superparamagnetic influence </b>
<br>
<br>
<hr>

In [ ]:
#generating more synthetic data

u_SD=0.07
u_MD=0.002
u_SP=0.00

I1,I2,I3,I4=1,0.5,1.0,1.5 #distinct values for I

k_zero=5e-2

SD=GGCD_1C(0.08,0.1,1,1,1,x) #SD component

MD=GGCD_1C(0.005,0.1,1,1,1,x) #MD component

SP=GGCD_1C(0.00,0.1,1,1,1,x) #SP component

#linear combinations of SD/MD +SP

SD_SP1=(SP)+(SD*I2)
SD_SP2=(SP)+(SD*I3)
SD_SP3=(SP)+(SD*I4)

SP_MD1=(SP)+(MD*I2)
SP_MD2=(SP)+(MD*I3)
SP_MD3=(SP)+(MD*I4)

#plotting the data

plt.rcParams.update({"text.usetex": True, "font.family": "serif", "font.size": 14})
figure,axes=plt.subplots(2,2,figsize=(10,10))

axes=axes.ravel()
axes[0].plot(x,SP,color='k',label='$SP$')
axes[0].plot(x,SD_SP1,label=f'$SP+SD,I={I2}$')
axes[0].plot(x,SD_SP2,label=f'$SP+SD,I={I3}$')
axes[0].plot(x,SD_SP3,label=f'$SP+SD,I={I4}$')
axes[0].grid(alpha=0.5)
axes[0].legend(fontsize=10,shadow=True,loc='upper left')
axes[0].set_ylabel('$\kappa$')
axes[0].set_xlabel('Applied field (T)')

axes[1].plot(x,np.gradient(SP,x),color='k',label='$SP$')
axes[1].plot(x,np.gradient(SD_SP1,x),label=f'$SP+SD,I={I2}$')
axes[1].plot(x,np.gradient(SD_SP2,x),label=f'$SP+SD,I={I3}$')
axes[1].plot(x,np.gradient(SD_SP3,x),label=f'$SP+SD,I={I4}$')
axes[1].grid(alpha=0.5)
axes[1].set_xlim(-0.5,0.5)
axes[1].legend(fontsize=10,shadow=True,loc='upper left')
axes[1].set_ylabel('$\partial{\kappa}/ \partial{} B$')
axes[1].set_xlabel('Applied field (T)')

axes[2].plot(x,SP,color='k',label='$SP$')
axes[2].plot(x,SP_MD1,label=f'$SP+MD,I={I2}$')
axes[2].plot(x,SP_MD2,label=f'$SP+MD,I={I3}$')
axes[2].plot(x,SP_MD3,label=f'$SP+MD,I={I4}$')
axes[2].grid(alpha=0.5)
axes[2].legend(fontsize=10,shadow=True,loc='upper left')
axes[2].set_ylabel('$\kappa$')
axes[2].set_xlabel('Applied field (T)')

axes[3].plot(x,np.gradient(SP,x),color='k',label='$SP$')
axes[3].plot(x,np.gradient(SP_MD1,x),label=f'$SP+MD,I={I2}$')
axes[3].plot(x,np.gradient(SP_MD2,x),label=f'$SP+MD,I={I3}$')
axes[3].plot(x,np.gradient(SP_MD3,x),label=f'$SP+MD,I={I4}$')
axes[3].set_xlim(-0.5,0.5)
axes[3].grid(alpha=0.5)
axes[3].legend(fontsize=10,shadow=True,loc='upper left')
axes[3].set_ylabel('$\partial{\kappa}/ \partial{} B$')
axes[3].set_xlabel('Applied field (T)')


Afont = {'family': 'serif',
        'backgroundcolor': 'white',
        'weight': 'normal',
        'size':11,
        }

at1 = AnchoredText(
    "a", prop=Afont, frameon=True, loc='upper right')
at1.patch.set_boxstyle("square,pad=0.")
axes[0].add_artist(at1)

at2 = AnchoredText(
    "b", prop=Afont, frameon=True, loc='upper right')
at2.patch.set_boxstyle("square,pad=0.")
axes[1].add_artist(at2)

at3 = AnchoredText(
    "c", prop=Afont, frameon=True, loc='upper right')
at3.patch.set_boxstyle("square,pad=0.")
axes[2].add_artist(at3)

at4 = AnchoredText(
    "d", prop=Afont, frameon=True, loc='upper right')
at4.patch.set_boxstyle("square,pad=0.")
axes[3].add_artist(at4)

figure.tight_layout()


<br>
<br>

<hr>
<br>
<br>
<b> High field saturation approach </b>
<br>
<br>
<hr>

In [ ]:
#generating synthetic data
x_hf=np.linspace(0.6,7,100)
high_field_test=high_field(-2.6,-2,x_hf)

#inversion procedure
error_hf,parameters_hf=Levenberg_Marquardt_HF(high_field,-1,-1.3,x_hf,high_field_test)

#calculating forward models from the optimized parameters

high_field_inv=high_field(parameters_hf[0],parameters_hf[1],x_hf)

#testing the inversion for a limited range of field values

xx,yy=[],[]

for i in range(np.size(x_hf)):
    if x_hf[i]<1:
        xx=np.append(xx,x_hf[i])
        yy=np.append(yy,high_field_test[i])

        
fi=interpolate.interp1d(xx,yy,kind='cubic')

x2 = np.linspace(xx[0],xx[-1],30) 
ya = fi(x2)

error_hf1,parameters_hf1=Levenberg_Marquardt_HF(high_field,-1,-1.3,xx,yy)
high_field_inv1=high_field(parameters_hf1[0],parameters_hf1[1],xx)


#plotting the data

figure,ax=plt.subplots(1,2,figsize=(10,5))
ax=ax.ravel()

ax[0].scatter(x_hf,high_field_test,label='Data',color='r',zorder=30,edgecolor='k')
ax[0].plot(x_hf,high_field_inv,label=f'$Inverted \ model,\lambda:{np.round(parameters_hf[0],3)},\phi:{np.round(parameters_hf[1],3)}$',zorder=20,color='k')
ax[0].grid(alpha=0.5)
ax[0].set_xlabel('Applied field (T)')
ax[0].set_ylabel('$\kappa_{hf}$')
ax[0].fill_betweenx(high_field_test+1,0.5, x_hf[10],color='royalblue',alpha=0.5,edgecolor='k')
ax[0].legend(shadow=True,fontsize=10,loc='upper left')
ax[0].set_ylim(0,max(high_field_test)+6)


# ax[1].scatter(x2,ya,label='Data',color='r',zorder=20)
ax[1].scatter(xx,yy,label='Data',color='r',zorder=30,edgecolor='k')
ax[1].plot(xx,high_field_inv1,label=f'$Inverted \ model,\lambda:{np.round(parameters_hf1[0],3)},\phi:{np.round(parameters_hf1[1],3)}$',color='k')
ax[1].grid(alpha=0.5)
ax[1].set_xlabel('Applied field (T)')
ax[1].set_ylabel('$\kappa_{hf}$')
ax[1].legend(shadow=True,fontsize=10,loc='upper left')
ax[1].set_ylim(0,max(high_field_test)+6)


Afont = {'family': 'serif',
        'backgroundcolor': 'white',
        'weight': 'normal',
        'size':11,
        }

at1 = AnchoredText(
    "a", prop=Afont, frameon=True, loc='upper right')
at1.patch.set_boxstyle("square,pad=0.")
ax[0].add_artist(at1)

at2 = AnchoredText(
    "b", prop=Afont, frameon=True, loc='upper right')
at2.patch.set_boxstyle("square,pad=0.")
ax[1].add_artist(at2)



figure.tight_layout()
